# Obligatorio - Machine Learning

# 1. Importación de librerías

In [ ]:
# Importaciones principales
import pandas as pd  # Manejo y análisis de datos estructurados (DataFrames)
import numpy as np  # Operaciones numéricas y manejo de arrays
import matplotlib.pyplot as plt  # Visualización de datos

# Preprocesamiento y selección de características
from sklearn.preprocessing import LabelEncoder, StandardScaler  # Codificación de variables categóricas y escalado de datos
from sklearn.feature_selection import f_regression, SelectKBest  # Selección de características relevantes basadas en correlación

# Modelos de aprendizaje supervisado
from sklearn.linear_model import LinearRegression, Ridge  # Modelos lineales para regresión (Ridge incluye regularización)
from sklearn.tree import DecisionTreeRegressor  # Árboles de decisión para regresión
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor  # Modelos avanzados: Random Forest, Gradient Boosting, AdaBoost
from sklearn.neural_network import MLPRegressor  # Redes neuronales para regresión

# Evaluación y optimización
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score  # Dividir datos, búsqueda de hiperparámetros y validación cruzada
from sklearn.metrics import mean_squared_error, r2_score  # Métricas de evaluación para modelos de regresión

# Text Mining
from sklearn.feature_extraction.text import TfidfVectorizer  # Representación numérica de texto (TF-IDF)

# Utilidades estadísticas y preprocesamiento avanzado
from scipy.stats.mstats import winsorize  # Manejo de valores extremos (outliers) mediante winsorización

# Herramientas de pipeline
from sklearn.pipeline import Pipeline  # Construcción de pipelines para flujos de preprocesamiento y modelado

# Visualización adicional
import matplotlib.pyplot as plt  # Gráficos adicionales para análisis visual

# Weights and Biases
import wandb  # Seguimiento, visualización y registro de métricas para experimentos de Machine Learning

import os
from math import sqrt

# Cargar datasets
df_train = pd.read_csv('./dataset/train.csv')  # Cargar el conjunto de entrenamiento
df_test = pd.read_csv('./dataset/test.csv')  # Cargar el conjunto de prueba

### Funciones Auxiliares

In [ ]:
# Paso 1: Rellenar valores faltantes
def clean_data(df):
    # Completar valores faltantes
    df['Year'] = df['Year'].fillna(df['Year'].median())  # Completar con la mediana
    df['Publisher'] = df['Publisher'].fillna('Unknown')  # Completar con "Unknown"

    # Eliminar columnas irrelevantes
    if 'Summary' in df.columns:
        df = df.drop(columns=['Summary'])

    return df

# Paso 2: Convertir columnas numéricas con valores inconsistentes
def convert_to_numeric(value):
    """
    Convierte valores como '1.2K' en valores numéricos.
    """
    value_str = str(value)
    if 'K' in value_str:
        return float(value_str.replace('K', '')) * 1000
    try:
        return float(value_str)
    except ValueError:
        return 0.0

# Paso 3: Manejar outliers
def handle_outliers(df, columns):
    """
    Aplica winsorización para manejar valores extremos en las columnas especificadas.
    """
    for col in columns:
        df[col] = winsorize(df[col], limits=[0.01, 0.01])  # Recorta 1% inferior y superior
    return df

# Paso 4: Codificar variables categóricas
def encode_categorical(df, categorical_columns):
    label_encoder = LabelEncoder()
    for col in categorical_columns:
        if col in df.columns:
            df[col] = label_encoder.fit_transform(df[col])
    return df

# Paso 5: Escalar columnas numéricas
def scale_numerical(df, numerical_columns):
    scaler = StandardScaler()
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
    return df

# Paso 6: Rellenar valores faltantes en `Genre` usando `Summary`
def fill_missing_genres(df):
    """
    Rellena los valores faltantes de la columna `Genre` basándose en la columna `Summary`.
    Utiliza un modelo de clasificación de texto.
    """
    # Separar filas con y sin `Genre`
    df_with_genre = df[df['Genre'].notnull()]
    df_missing_genre = df[df['Genre'].isnull()]

    if len(df_missing_genre) > 0:
        # Convertir valores categóricos de `Genre` a números
        label_encoder = LabelEncoder()
        df_with_genre['Genre'] = label_encoder.fit_transform(df_with_genre['Genre'])

        # Modelo de clasificación de texto
        text_clf = Pipeline([
            ('tfidf', TfidfVectorizer(stop_words='english', max_features=1000)),
            ('clf', LogisticRegression(max_iter=1000, random_state=42))
        ])

        # Entrenar el modelo con los datos disponibles
        text_clf.fit(df_with_genre['Summary'], df_with_genre['Genre'])

        # Predecir los géneros faltantes
        predicted_genres = text_clf.predict(df_missing_genre['Summary'])

        # Convertir las predicciones de vuelta a etiquetas originales
        df_missing_genre['Genre'] = label_encoder.inverse_transform(predicted_genres)

        # Combinar las filas con `Genre` y las predichas
        df = pd.concat([df_with_genre, df_missing_genre])

# Paso 7: Crear nuevas columnas derivadas
def create_new_columns(df):
    """
    Crea nuevas columnas derivadas basadas en la información existente.
    """
    df['Game_Age'] = 2024 - df['Year']
    
    # Ratios regionales
    df['NorthAmerica_Global_Ratio'] = df['North America'] / df['Global']
    df['Europe_Global_Ratio'] = df['Europe'] / df['Global']
    df['Japan_Global_Ratio'] = df['Japan'] / df['Global']
    df['RestOfWorld_Global_Ratio'] = df['Rest of World'] / df['Global']

    # Relación entre reseñas y wishlist
    df['Reviews_Wishlist_Ratio'] = df['Number of Reviews'] / df['Wishlist']

    # Interacciones categóricas
    df['Publisher_Platform_Interaction'] = df['Publisher'] + df['Platform']

    return df


# Preprocesamiento de datos

In [ ]:
def preprocess_data_with_genre_and_columns(df):
    # Limpiar datos
    df = clean_data(df)

    # Rellenar valores faltantes en `Genre` usando `Summary`
    if 'Summary' in df.columns and 'Genre' in df.columns:
        df = fill_missing_genres(df)

    # Convertir columnas numéricas con valores inconsistentes
    columns_to_convert = ['Europe', 'Japan', 'Rest of World', 'North America', 
                          'Global', 'Number of Reviews', 'Wishlist']
    for col in columns_to_convert:
        if col in df.columns:
            df[col] = df[col].apply(convert_to_numeric)

    # Manejar outliers
    outlier_columns = ['Global', 'Wishlist']
    df = handle_outliers(df, outlier_columns)

    # Codificar columnas categóricas
    categorical_columns = ['Game Title', 'Publisher', 'Platform', 'Genre']
    df = encode_categorical(df, categorical_columns)

    # Crear nuevas columnas derivadas
    df = create_new_columns(df)

    # Escalar columnas numéricas
    numerical_columns = ['North America', 'Europe', 'Japan', 'Rest of World', 
                         'Global', 'Number of Reviews', 'Wishlist', 'Game_Age', 
                         'Europe_Global_Ratio', 'Japan_Global_Ratio', 
                         'NorthAmerica_Global_Ratio', 'RestOfWorld_Global_Ratio',
                         'Reviews_Wishlist_Ratio']
    df = scale_numerical(df, numerical_columns)

    # Eliminar columnas irrelevantes
    columns_to_drop = ['Game Title']
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

    # Eliminar las columnas regionales originales después de crear los ratios
    columns_to_drop = ['North America', 'Europe', 'Japan', 'Rest of World']
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

    return df

# Aplicar el preprocesamiento actualizado
df_train = preprocess_data_with_genre_and_columns(df_train)
df_test = preprocess_data_with_genre_and_columns(df_test)

# Verificar el resultado
print("Datos preprocesados con géneros rellenados y nuevas columnas (train):")
print(df_train.head())

# Analisis exploratorio

In [ ]:
# Función para evaluar correlación con la variable objetivo (para regresión)
def evaluate_feature_correlation(X, y):
    """
    Evalúa la correlación de cada característica con la variable objetivo.
    Devuelve un DataFrame con las puntuaciones.
    """
    selector = SelectKBest(score_func=f_regression, k='all')
    selector.fit(X, y)
    scores = selector.scores_

    # Crear un DataFrame con las puntuaciones
    feature_scores = pd.DataFrame({'Feature': X.columns, 'Score': scores})
    feature_scores = feature_scores.sort_values(by='Score', ascending=False)

    print("Importancia de características basada en correlación:")
    print(feature_scores)
    return feature_scores

# Función para evaluar la importancia de características usando un modelo
def evaluate_feature_importance_model(X, y):
    """
    Evalúa la importancia de características utilizando un RandomForestRegressor.
    Devuelve un DataFrame con las importancias.
    """
    model = RandomForestRegressor(random_state=42)
    model.fit(X, y)
    importances = model.feature_importances_

    # Crear un DataFrame con las importancias
    feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
    feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

    # Visualizar la importancia de características
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importances['Feature'], feature_importances['Importance'])
    plt.xlabel("Importancia")
    plt.ylabel("Características")
    plt.title("Importancia de características según RandomForest")
    plt.gca().invert_yaxis()
    plt.show()

    print("Importancia de características basada en RandomForest:")
    print(feature_importances)
    return feature_importances

# Selección de las características más relevantes
def select_top_features(X, y, k=10):
    """
    Selecciona las k características más relevantes utilizando SelectKBest.
    """
    selector = SelectKBest(score_func=f_regression, k=k)
    X_selected = selector.fit_transform(X, y)

    # Obtener las columnas seleccionadas
    selected_columns = X.columns[selector.get_support()]
    print(f"Características seleccionadas ({k} mejores): {selected_columns}")
    return X_selected, selected_columns

# Para df_train (como ya tienes)
X_train = df_train.drop(columns=['Rating'])  # Variables predictoras
y_train = df_train['Rating']  # Variable objetivo

# Evaluar y seleccionar características en df_train
correlation_scores = evaluate_feature_correlation(X_train, y_train)
importance_scores = evaluate_feature_importance_model(X_train, y_train)
X_train_selected, selected_columns = select_top_features(X_train, y_train, k=10)

# Actualizar el dataset de entrenamiento
df_train_selected = pd.DataFrame(X_train_selected, columns=selected_columns)
df_train_selected['Rating'] = y_train  # Añadir la columna objetivo para usar en el modelo

# Aplicar la misma selección de características al conjunto de prueba**
if 'Rating' in df_test.columns:
    X_test = df_test.drop(columns=['Rating'])  # Variables predictoras en el conjunto de prueba
    y_test = df_test['Rating']  # Variable objetivo en el conjunto de prueba
else:
    X_test = df_test
    y_test = None

# Seleccionar las mismas características en df_test usando `selected_columns`
X_test_selected = X_test[selected_columns]

# Actualizar el dataset de prueba
df_test_selected = pd.DataFrame(X_test_selected, columns=selected_columns)
if y_test is not None:
    df_test_selected['Rating'] = y_test  # Solo si `Rating` está disponible

### Proceso de nuevo los datos eliminando columnas irrelevantes

In [ ]:
def preprocess_data_optimized(df):

    # Eliminar columnas menos relevantes
    columns_to_drop = ['id', 'Publisher_Platform_Interaction', 'Publisher', 'Wishlist', 
                       'Number of Reviews', 'Reviews_Wishlist_Ratio']
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

    return df

# Aplicar el preprocesamiento optimizado
df_train = preprocess_data_optimized(df_train)
df_test = preprocess_data_optimized(df_test)

# Verificar el resultado
print("Datos preprocesados optimizados (train):")
print(df_train.head())

In [ ]:

# Función para evaluar correlación con la variable objetivo (para regresión)
def evaluate_feature_correlation(X, y):
    """
    Evalúa la correlación de cada característica con la variable objetivo.
    Devuelve un DataFrame con las puntuaciones.
    """
    selector = SelectKBest(score_func=f_regression, k='all')
    selector.fit(X, y)
    scores = selector.scores_

    # Crear un DataFrame con las puntuaciones
    feature_scores = pd.DataFrame({'Feature': X.columns, 'Score': scores})
    feature_scores = feature_scores.sort_values(by='Score', ascending=False)

    print("Importancia de características basada en correlación:")
    print(feature_scores)
    return feature_scores

# Función para evaluar la importancia de características usando un modelo
def evaluate_feature_importance_model(X, y):
    """
    Evalúa la importancia de características utilizando un RandomForestRegressor.
    Devuelve un DataFrame con las importancias.
    """
    model = RandomForestRegressor(random_state=42)
    model.fit(X, y)
    importances = model.feature_importances_

    # Crear un DataFrame con las importancias
    feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
    feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

    # Visualizar la importancia de características
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importances['Feature'], feature_importances['Importance'])
    plt.xlabel("Importancia")
    plt.ylabel("Características")
    plt.title("Importancia de características según RandomForest")
    plt.gca().invert_yaxis()
    plt.show()

    print("Importancia de características basada en RandomForest:")
    print(feature_importances)
    return feature_importances

# Selección de las características más relevantes
def select_top_features(X, y, k=10):
    """
    Selecciona las k características más relevantes utilizando SelectKBest.
    """
    selector = SelectKBest(score_func=f_regression, k=k)
    X_selected = selector.fit_transform(X, y)

    # Obtener las columnas seleccionadas
    selected_columns = X.columns[selector.get_support()]
    print(f"Características seleccionadas ({k} mejores): {selected_columns}")
    return X_selected, selected_columns

# Para df_train (como ya tienes)
X_train = df_train.drop(columns=['Rating'])  # Variables predictoras
y_train = df_train['Rating']  # Variable objetivo

# Evaluar y seleccionar características en df_train
correlation_scores = evaluate_feature_correlation(X_train, y_train)
importance_scores = evaluate_feature_importance_model(X_train, y_train)
X_train_selected, selected_columns = select_top_features(X_train, y_train, k=10)

# Actualizar el dataset de entrenamiento
df_train_selected = pd.DataFrame(X_train_selected, columns=selected_columns)
df_train_selected['Rating'] = y_train  # Añadir la columna objetivo para usar en el modelo

# Aplicar la misma selección de características al conjunto de prueba
# Como no tiene la columna `Rating`, simplemente seleccionamos las columnas relevantes
X_test = df_test[selected_columns]

# Actualizar el dataset de prueba
df_test_selected = pd.DataFrame(X_test, columns=selected_columns)

# df_test_selected ahora contiene solo las características seleccionadas para hacer predicciones
print("Características seleccionadas en el conjunto de prueba:")
print(df_test_selected.head())

# Entrenamiento y Evaluacion de los modelos

In [ ]:
# Dividir los datos para entrenamiento y prueba

# Entrenamiento: X_train y y_train
X_train = df_train_selected.drop(columns=['Rating'])  # Variables predictoras del conjunto de entrenamiento
y_train = df_train_selected['Rating']  # Variable objetivo del conjunto de entrenamiento

# Prueba: X_test (sin Rating)
X_test = df_test_selected  # df_test seleccionado y procesado ya no incluye la columna Rating

Regresion Lineal

In [ ]:
# Crear carpetas base para resultados
os.makedirs("predictions/RegresionLineal", exist_ok=True)
os.makedirs("metrics/RegresionLineal", exist_ok=True)
os.makedirs("metrics/MejoresModelos", exist_ok=True)

# Función para entrenar y evaluar el modelo

def train_and_evaluate_lr(X_train, y_train, X_test):
    # Escalar los datos
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Configuración de hiperparámetros
    param_grid_lr = {
        'fit_intercept': [True, False],  # Ajustar o no el intercepto
    }

    # Entrenamiento y búsqueda de hiperparámetros
    grid_search_lr = GridSearchCV(
        LinearRegression(),
        param_grid_lr,
        cv=5,
        scoring='neg_mean_squared_error',
        verbose=1,
        n_jobs=-1
    )

    grid_search_lr.fit(X_train_scaled, y_train)
    best_model = grid_search_lr.best_estimator_
    print(f"Mejores parámetros: {grid_search_lr.best_params_}")

    # Validación cruzada en el conjunto de entrenamiento
    cv_scores = cross_val_score(best_model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_mean = -cv_scores.mean()
    cv_std = cv_scores.std()
    cv_rmse = sqrt(cv_mean)

    print("\nValidación cruzada:")
    print(f"MSE promedio (CV): {cv_mean:.4f}")
    print(f"RMSE promedio (CV): {cv_rmse:.4f}")
    print(f"Desviación estándar (MSE): {cv_std:.4f}")

    # Generar predicciones para el conjunto de prueba
    y_pred = best_model.predict(X_test_scaled)

    # Guardar predicciones en archivo CSV dentro de la carpeta `predictions/RegresionLineal`
    predictions_file = f"predictions/RegresionLineal/predicciones_regresion_lineal.csv"
    pd.DataFrame({"Id": range(len(y_pred)), "Predicted_Rating": y_pred}).to_csv(predictions_file, index=False)
    print(f"Predicciones guardadas en {predictions_file}")

    # Guardar métricas en archivo CSV dentro de `metrics/RegresionLineal`
    metrics_file = f"metrics/RegresionLineal/metrics_regresion_lineal.csv"
    with open(metrics_file, "w") as f:
        f.write("Métrica,Valor\n")
        f.write(f"MSE promedio (CV),{cv_mean:.4f}\n")
        f.write(f"RMSE promedio (CV),{cv_rmse:.4f}\n")
        f.write(f"Desviación estándar (MSE),{cv_std:.4f}\n")
    print(f"Métricas guardadas en {metrics_file}")

    return cv_mean, cv_std, metrics_file, best_model

# Proceso principal para obtener el mejor modelo
def main_lr(X_train, y_train, X_test):
    cv_mean, cv_std, metrics_file, best_model = train_and_evaluate_lr(X_train, y_train, X_test)

    # Guardar métricas del mejor modelo en archivo CSV
    best_metrics_file = "metrics/MejoresModelos/LR_mejor_modelo.csv"
    with open(best_metrics_file, "w") as f:
        f.write("Métrica,Valor\n")
        f.write(f"MSE promedio (CV),{cv_mean:.4f}\n")
        f.write(f"RMSE promedio (CV),{sqrt(cv_mean):.4f}\n")
        f.write(f"Desviación estándar (MSE),{cv_std:.4f}\n")
    print(f"Métricas del mejor modelo guardadas en {best_metrics_file}")

    return best_model

# Llamar al proceso principal
best_model = main_lr(X_train, y_train, X_test)


Con regularizacion ridge

In [ ]:
# Crear carpetas base para resultados
os.makedirs("predictions/RidgeRegression", exist_ok=True)
os.makedirs("metrics/RidgeRegression", exist_ok=True)
os.makedirs("metrics/MejoresModelos", exist_ok=True)

def train_and_evaluate_ridge(X_train, y_train, X_test, alpha, fit_intercept):
    # Configuración de hiperparámetros
    param_grid_ridge = {
        'alpha': [alpha],  # Regularización L2
        'fit_intercept': [fit_intercept]
    }

    # Modelo base Ridge
    ridge_model = Ridge()

    # Optimización con GridSearchCV
    grid_search_ridge = GridSearchCV(
        estimator=ridge_model,
        param_grid=param_grid_ridge,
        cv=5,
        scoring='neg_mean_squared_error',
        verbose=1,
        n_jobs=-1
    )

    # Entrenar el modelo con los datos de entrenamiento
    grid_search_ridge.fit(X_train, y_train)
    best_model = grid_search_ridge.best_estimator_

    print(f"Mejores parámetros para alpha={alpha}, fit_intercept={fit_intercept}: {grid_search_ridge.best_params_}")

    # Validación cruzada
    cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_mean = -cv_scores.mean()
    cv_std = cv_scores.std()
    cv_rmse = np.sqrt(cv_mean)

    print("\nValidación cruzada:")
    print(f"MSE promedio (CV): {cv_mean:.4f}")
    print(f"RMSE promedio (CV): {cv_rmse:.4f}")
    print(f"Desviación estándar (MSE): {cv_std:.4f}")

    # Predicciones en el conjunto de prueba
    y_pred = best_model.predict(X_test)  # Sin limitación de resultados

    # Guardar predicciones
    predictions_file = f"predictions/RidgeRegression/predicciones_alpha_{alpha}_fit_intercept_{fit_intercept}.csv"
    pd.DataFrame({"Id": range(len(y_pred)), "Predicted_Rating": y_pred}).to_csv(predictions_file, index=False)
    print(f"Predicciones guardadas en {predictions_file}")

    # Guardar métricas
    metrics_file = f"metrics/RidgeRegression/metrics_alpha_{alpha}_fit_intercept_{fit_intercept}.csv"
    with open(metrics_file, "w") as f:
        f.write("Métrica,Valor\n")
        f.write(f"MSE promedio (CV),{cv_mean:.4f}\n")
        f.write(f"RMSE promedio (CV),{cv_rmse:.4f}\n")
        f.write(f"Desviación estándar (MSE),{cv_std:.4f}\n")
    print(f"Métricas guardadas en {metrics_file}")

    return cv_mean, cv_std, metrics_file, best_model

def main_ridge(X_train, y_train, X_test):
    results = []
    alpha_values = [0.1, 1, 10]
    fit_intercept_values = [True, False]

    # Iterar sobre diferentes combinaciones de hiperparámetros
    for alpha in alpha_values:
        for fit_intercept in fit_intercept_values:
            cv_mean, cv_std, metrics_file, model = train_and_evaluate_ridge(X_train, y_train, X_test, alpha, fit_intercept)
            results.append((alpha, fit_intercept, cv_mean, cv_std, metrics_file, model))

    # Seleccionar el mejor modelo basado en MSE promedio
    best_model = min(results, key=lambda x: x[2])
    print("\nMejor modelo:")
    print(f"alpha: {best_model[0]}, fit_intercept: {best_model[1]}, MSE promedio: {best_model[2]:.4f}")

    # Guardar métricas del mejor modelo en archivo CSV
    best_metrics_file = "metrics/MejoresModelos/Ridge_mejor_modelo.csv"
    with open(best_metrics_file, "w") as f:
        f.write("Métrica,Valor\n")
        f.write(f"MSE promedio (CV),{best_model[2]:.4f}\n")
        f.write(f"RMSE promedio (CV),{np.sqrt(best_model[2]):.4f}\n")
        f.write(f"Desviación estándar (MSE),{best_model[3]:.4f}\n")
    print(f"Métricas del mejor modelo guardadas en {best_metrics_file}")

    return best_model

# Llamar al proceso principal
best_model = main_ridge(X_train, y_train, X_test)

Arboles de decision

In [ ]:
# Crear carpetas base para resultados
os.makedirs("predictions/ArbolesDeDecision", exist_ok=True)
os.makedirs("metrics/ArbolesDeDecision", exist_ok=True)
os.makedirs("metrics/MejoresModelos", exist_ok=True)

# Función para entrenar y evaluar un modelo con un conjunto de hiperparámetros
def train_and_evaluate_tree(X_train, y_train, X_test, max_depth, criterion, max_features):
    # Configuración de hiperparámetros
    param_grid_tree = {
        'max_depth': [max_depth],
        'criterion': [criterion],
        'max_features': [max_features],
        'min_samples_leaf': [5, 10],
        'min_impurity_decrease': [0.0],  # Valor fijo para este caso
        'splitter': ['best']
    }
    
    # Entrenamiento y búsqueda de hiperparámetros
    grid_search_tree = GridSearchCV(
        DecisionTreeRegressor(random_state=42),
        param_grid_tree,
        cv=5,
        scoring='neg_mean_squared_error',
        verbose=1,
        n_jobs=-1
    )
    
    grid_search_tree.fit(X_train, y_train)
    best_model = grid_search_tree.best_estimator_
    print(f"Mejores parámetros para max_depth={max_depth}, criterion={criterion}, max_features={max_features}: {grid_search_tree.best_params_}")
    
    # Validación cruzada en el conjunto de entrenamiento
    cv_scores_mse = cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_mean_mse = -cv_scores_mse.mean()
    cv_std_mse = cv_scores_mse.std()
    cv_rmse = sqrt(cv_mean_mse)
    
    print("\nValidación cruzada:")
    print(f"MSE promedio (CV): {cv_mean_mse:.4f}")
    print(f"RMSE promedio (CV): {cv_rmse:.4f}")
    print(f"Desviación estándar (MSE): {cv_std_mse:.4f}")
    
    # Generar predicciones para el conjunto de prueba
    y_pred = best_model.predict(X_test)
    
    # Guardar predicciones en archivo CSV dentro de la carpeta `predictions/ArbolesDeDecision`
    predictions_file = f"predictions/ArbolesDeDecision/predicciones_max_depth_{max_depth}_criterion_{criterion}_features_{max_features}.csv"
    pd.DataFrame({"Id": range(len(y_pred)), "Predicted_Rating": y_pred}).to_csv(predictions_file, index=False)
    print(f"Predicciones guardadas en {predictions_file}")
    
    # Guardar métricas en archivo CSV dentro de `metrics/ArbolesDeDecision`
    metrics_file = f"metrics/ArbolesDeDecision/metrics_max_depth_{max_depth}_criterion_{criterion}_features_{max_features}.csv"
    with open(metrics_file, "w") as f:
        f.write("Métrica,Valor\n")
        f.write(f"MSE promedio (CV),{cv_mean_mse:.4f}\n")
        f.write(f"RMSE promedio (CV),{cv_rmse:.4f}\n")
        f.write(f"Desviación estándar (MSE),{cv_std_mse:.4f}\n")
    print(f"Métricas guardadas en {metrics_file}")
    
    return cv_mean_mse, cv_std_mse, metrics_file, best_model

# Proceso principal para iterar sobre los hiperparámetros
def main_tree(X_train, y_train, X_test):
    results = []
    max_depth_values = [3, 7, 15]
    criterion_values = ['squared_error', 'absolute_error']  # Cambiar a criterios válidos
    max_features_values = ['sqrt', 'log2']
    
    # Iterar sobre diferentes combinaciones de hiperparámetros
    for max_depth in max_depth_values:
        for criterion in criterion_values:
            for max_features in max_features_values:
                cv_mean_mse, cv_std_mse, metrics_file, model = train_and_evaluate_tree(
                    X_train, y_train, X_test, max_depth, criterion, max_features
                )
                results.append((max_depth, criterion, max_features, cv_mean_mse, cv_std_mse, metrics_file, model))
    
    # Seleccionar el mejor modelo basado en MSE promedio
    best_model = min(results, key=lambda x: x[3])
    print("\nMejor modelo:")
    print(f"max_depth: {best_model[0]}, criterion: {best_model[1]}, max_features: {best_model[2]}, MSE promedio: {best_model[3]:.4f}")
    
    # Guardar métricas del mejor modelo en archivo CSV
    best_metrics_file = "metrics/MejoresModelos/DT_mejor_modelo.csv"
    with open(best_metrics_file, "w") as f:
        f.write("Métrica,Valor\n")
        f.write(f"MSE promedio (CV),{best_model[3]:.4f}\n")
        f.write(f"RMSE promedio (CV),{sqrt(best_model[3]):.4f}\n")
        f.write(f"Desviación estándar (MSE),{best_model[4]:.4f}\n")
    print(f"Métricas del mejor modelo guardadas en {best_metrics_file}")
    
    return best_model

# Llamar al proceso principal
best_model = main_tree(X_train, y_train, X_test)

Random Forest

In [ ]:
# Crear carpetas base para resultados
os.makedirs("predictions/RandomForest", exist_ok=True)
os.makedirs("metrics/RandomForest", exist_ok=True)
os.makedirs("metrics/MejoresModelos", exist_ok=True)

# Función para entrenar y evaluar un modelo con un conjunto de hiperparámetros
def train_and_evaluate_rf(X_train, y_train, X_test, n_estimators, max_depth):
    # Configuración de hiperparámetros
    param_grid_rf = {
        'n_estimators': [n_estimators],
        'max_depth': [max_depth],
        'min_samples_split': [2],
        'min_samples_leaf': [1]
    }

    # Entrenamiento y búsqueda de hiperparámetros
    grid_search_rf = GridSearchCV(
        RandomForestRegressor(random_state=42),
        param_grid_rf,
        cv=5,
        scoring='neg_mean_squared_error',
        verbose=1,
        n_jobs=-1
    )

    grid_search_rf.fit(X_train, y_train)
    best_model = grid_search_rf.best_estimator_
    print(f"Mejores parámetros para n_estimators={n_estimators}, max_depth={max_depth}: {grid_search_rf.best_params_}")

    # Validación cruzada en el conjunto de entrenamiento
    cv_scores_mse = cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_mean_mse = -cv_scores_mse.mean()
    cv_std_mse = cv_scores_mse.std()
    cv_rmse = sqrt(cv_mean_mse)

    print("\nValidación cruzada:")
    print(f"MSE promedio (CV): {cv_mean_mse:.4f}")
    print(f"RMSE promedio (CV): {cv_rmse:.4f}")
    print(f"Desviación estándar (MSE): {cv_std_mse:.4f}")

    # Generar predicciones para el conjunto de prueba
    y_pred = best_model.predict(X_test)

    # Guardar predicciones en archivo CSV dentro de la carpeta `predictions/RandomForest`
    predictions_file = f"predictions/RandomForest/predicciones_n_estimators_{n_estimators}_max_depth_{max_depth}.csv"
    pd.DataFrame({"Id": range(len(y_pred)), "Predicted_Rating": y_pred}).to_csv(predictions_file, index=False)
    print(f"Predicciones guardadas en {predictions_file}")

    # Guardar métricas en archivo CSV dentro de `metrics/RandomForest`
    metrics_file = f"metrics/RandomForest/metrics_n_estimators_{n_estimators}_max_depth_{max_depth}.csv"
    with open(metrics_file, "w") as f:
        f.write("Métrica,Valor\n")
        f.write(f"MSE promedio (CV),{cv_mean_mse:.4f}\n")
        f.write(f"RMSE promedio (CV),{cv_rmse:.4f}\n")
        f.write(f"Desviación estándar (MSE),{cv_std_mse:.4f}\n")
    print(f"Métricas guardadas en {metrics_file}")

    return cv_mean_mse, cv_std_mse, metrics_file, best_model

# Proceso principal para iterar sobre los hiperparámetros
def main_rf(X_train, y_train, X_test):
    results = []
    n_estimators_values = [50, 100]
    max_depth_values = [5, 10, 20]

    # Iterar sobre diferentes combinaciones de n_estimators y max_depth
    for n_estimators in n_estimators_values:
        for max_depth in max_depth_values:
            cv_mean_mse, cv_std_mse, metrics_file, model = train_and_evaluate_rf(X_train, y_train, X_test, n_estimators, max_depth)
            results.append((n_estimators, max_depth, cv_mean_mse, cv_std_mse, metrics_file, model))

    # Seleccionar el mejor modelo basado en MSE promedio
    best_model = min(results, key=lambda x: x[2])
    print("\nMejor modelo:")
    print(f"n_estimators: {best_model[0]}, max_depth: {best_model[1]}, MSE promedio: {best_model[2]:.4f}")

    # Guardar métricas del mejor modelo en archivo CSV
    best_metrics_file = "metrics/MejoresModelos/RF_mejor_modelo.csv"
    with open(best_metrics_file, "w") as f:
        f.write("Métrica,Valor\n")
        f.write(f"MSE promedio (CV),{best_model[2]:.4f}\n")
        f.write(f"RMSE promedio (CV),{sqrt(best_model[2]):.4f}\n")
        f.write(f"Desviación estándar (MSE),{best_model[3]:.4f}\n")
    print(f"Métricas del mejor modelo guardadas en {best_metrics_file}")

    return best_model

# Llamar al proceso principal
best_model = main_rf(X_train, y_train, X_test)


Gradient Boosting

In [ ]:
# Crear carpetas base para resultados
os.makedirs("predictions/GradientBoosting", exist_ok=True)
os.makedirs("metrics/GradientBoosting", exist_ok=True)
os.makedirs("metrics/MejoresModelos", exist_ok=True)

# Función para entrenar y evaluar un modelo con un conjunto de hiperparámetros
def train_and_evaluate_gb(X_train, y_train, X_test, n_estimators, learning_rate):
    # Configuración de hiperparámetros
    param_grid_gb = {
        'n_estimators': [n_estimators],
        'learning_rate': [learning_rate],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
    }
    
    # Entrenamiento y búsqueda de hiperparámetros
    grid_search_gb = GridSearchCV(
        GradientBoostingRegressor(random_state=42),
        param_grid_gb,
        cv=5,
        scoring='neg_mean_squared_error',
        verbose=1,
        n_jobs=-1
    )
    
    grid_search_gb.fit(X_train, y_train)
    best_model = grid_search_gb.best_estimator_
    print(f"Mejores parámetros para n_estimators={n_estimators}, learning_rate={learning_rate}: {grid_search_gb.best_params_}")
    
    # Validación cruzada en el conjunto de entrenamiento
    cv_scores_mse = cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_mean_mse = -cv_scores_mse.mean()
    cv_std_mse = cv_scores_mse.std()
    cv_rmse = sqrt(cv_mean_mse)
    
    print("\nValidación cruzada:")
    print(f"MSE promedio (CV): {cv_mean_mse:.4f}")
    print(f"RMSE promedio (CV): {cv_rmse:.4f}")
    print(f"Desviación estándar (MSE): {cv_std_mse:.4f}")
    
    # Generar predicciones para el conjunto de prueba
    y_pred = best_model.predict(X_test)
    
    # Guardar predicciones en archivo CSV dentro de la carpeta `predictions/GradientBoosting`
    predictions_file = f"predictions/GradientBoosting/predicciones_n_estimators_{n_estimators}_lr_{learning_rate}.csv"
    pd.DataFrame({"Id": range(len(y_pred)), "Predicted_Rating": y_pred}).to_csv(predictions_file, index=False)
    print(f"Predicciones guardadas en {predictions_file}")
    
    # Guardar métricas en archivo CSV dentro de `metrics/GradientBoosting`
    metrics_file = f"metrics/GradientBoosting/metrics_n_estimators_{n_estimators}_lr_{learning_rate}.csv"
    metrics_table = pd.DataFrame({
        "Métrica": ["MSE promedio (CV)", "RMSE promedio (CV)", "Desviación estándar (MSE)"],
        "Valor": [cv_mean_mse, cv_rmse, cv_std_mse]
    })
    metrics_table.to_csv(metrics_file, index=False)
    print(f"Métricas guardadas en {metrics_file}")
    
    return cv_mean_mse, metrics_file, best_model

# Proceso principal para iterar sobre los hiperparámetros
def main_gb(X_train, y_train, X_test):
    results = []
    n_estimators_values = [50, 100, 200]
    learning_rate_values = [0.01, 0.1, 0.2]
    
    # Iterar sobre diferentes combinaciones de n_estimators y learning_rate
    for n_estimators in n_estimators_values:
        for learning_rate in learning_rate_values:
            cv_mean_mse, metrics_file, model = train_and_evaluate_gb(
                X_train, y_train, X_test, n_estimators, learning_rate
            )
            results.append((n_estimators, learning_rate, cv_mean_mse, metrics_file, model))
    
    # Seleccionar el mejor modelo basado en MSE promedio
    best_model = min(results, key=lambda x: x[2])
    print("\nMejor modelo:")
    print(f"n_estimators: {best_model[0]}, learning_rate: {best_model[1]}, MSE promedio: {best_model[2]:.4f}")
    
    # Guardar métricas del mejor modelo en `metrics/MejoresModelos/GB_mejor_modelo.csv`
    best_metrics_file = "metrics/MejoresModelos/GB_mejor_modelo.csv"
    pd.read_csv(best_model[3]).to_csv(best_metrics_file, index=False)
    print(f"Métricas del mejor modelo guardadas en {best_metrics_file}")
    
    return best_model

# Llamar al proceso principal
best_model = main_gb(X_train, y_train, X_test)

Ada boost

In [ ]:
# Crear carpetas base para resultados
os.makedirs("predictions/AdaBoost", exist_ok=True)
os.makedirs("metrics/AdaBoost", exist_ok=True)
os.makedirs("metrics/MejoresModelos", exist_ok=True)

# Función para entrenar y evaluar un modelo con un conjunto de hiperparámetros
def train_and_evaluate_ada(X_train, y_train, X_test, n_estimators, learning_rate):
    # Configuración de hiperparámetros
    param_grid_ada = {
        'n_estimators': [n_estimators],  # Número de árboles
        'learning_rate': [learning_rate],  # Tasa de aprendizaje
        'estimator__max_depth': [5, 7, 10]  # No cambiar esta fila
    }
    
    # Modelo base de AdaBoost con DecisionTreeRegressor
    base_estimator = DecisionTreeRegressor(random_state=42)
    ada_model = AdaBoostRegressor(estimator=base_estimator, random_state=42)
    
    # Optimización con GridSearchCV
    grid_search_ada = GridSearchCV(
        estimator=ada_model,
        param_grid=param_grid_ada,
        cv=5,
        scoring='neg_mean_squared_error',
        verbose=1,
        n_jobs=-1
    )
    
    grid_search_ada.fit(X_train, y_train)
    best_model = grid_search_ada.best_estimator_
    print(f"Mejores parámetros para n_estimators={n_estimators}, learning_rate={learning_rate}: {grid_search_ada.best_params_}")
    
    # Validación cruzada en el conjunto de entrenamiento
    cv_scores_mse = cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_mean_mse = -cv_scores_mse.mean()
    cv_std_mse = cv_scores_mse.std()
    cv_rmse = np.sqrt(cv_mean_mse)
    
    print("\nValidación cruzada:")
    print(f"MSE promedio (CV): {cv_mean_mse:.4f}")
    print(f"RMSE promedio (CV): {cv_rmse:.4f}")
    print(f"Desviación estándar (MSE): {cv_std_mse:.4f}")
    
    # Generar predicciones para el conjunto de prueba
    y_pred = best_model.predict(X_test)
    
    # Guardar predicciones en archivo CSV dentro de la carpeta `predictions/AdaBoost`
    predictions_file = f"predictions/AdaBoost/predicciones_n_estimators_{n_estimators}_lr_{learning_rate}.csv"
    pd.DataFrame({"Id": range(len(y_pred)), "Predicted_Rating": y_pred}).to_csv(predictions_file, index=False)
    print(f"Predicciones guardadas en {predictions_file}")
    
    # Guardar métricas en archivo CSV dentro de `metrics/AdaBoost`
    metrics_file = f"metrics/AdaBoost/metrics_n_estimators_{n_estimators}_lr_{learning_rate}.csv"
    metrics_table = pd.DataFrame({
        "Métrica": ["MSE promedio (CV)", "RMSE promedio (CV)", "Desviación estándar (MSE)"],
        "Valor": [cv_mean_mse, cv_rmse, cv_std_mse]
    })
    metrics_table.to_csv(metrics_file, index=False)
    print(f"Métricas guardadas en {metrics_file}")
    
    return cv_mean_mse, metrics_file, best_model

# Proceso principal para iterar sobre los hiperparámetros
def main_ada(X_train, y_train, X_test):
    results = []
    n_estimators_values = [50, 100, 200]
    learning_rate_values = [0.01, 0.1, 0.2]
    
    # Iterar sobre diferentes combinaciones de n_estimators y learning_rate
    for n_estimators in n_estimators_values:
        for learning_rate in learning_rate_values:
            cv_mean_mse, metrics_file, model = train_and_evaluate_ada(
                X_train, y_train, X_test, n_estimators, learning_rate
            )
            results.append((n_estimators, learning_rate, cv_mean_mse, metrics_file, model))
    
    # Seleccionar el mejor modelo basado en MSE promedio
    best_model = min(results, key=lambda x: x[2])
    print("\nMejor modelo:")
    print(f"n_estimators: {best_model[0]}, learning_rate: {best_model[1]}, MSE promedio: {best_model[2]:.4f}")
    
    # Guardar métricas del mejor modelo en `metrics/MejoresModelos/AdaBoost_mejor_modelo.csv`
    best_metrics_file = "metrics/MejoresModelos/AdaBoost_mejor_modelo.csv"
    pd.read_csv(best_model[3]).to_csv(best_metrics_file, index=False)
    print(f"Métricas del mejor modelo guardadas en {best_metrics_file}")
    
    return best_model

# Llamar al proceso principal
best_model = main_ada(X_train, y_train, X_test)

Redes Neuronales

In [ ]:
# Crear carpetas base para resultados
os.makedirs("predictions/RedesNeuronales", exist_ok=True)
os.makedirs("metrics/RedesNeuronales", exist_ok=True)
os.makedirs("metrics/MejoresModelos", exist_ok=True)

# Función para entrenar y evaluar Redes Neuronales
def train_and_evaluate_nn(X_train, y_train, X_test, hidden_layer_sizes, learning_rate_init):
    # Configuración de hiperparámetros
    param_grid_mlp = {
        'hidden_layer_sizes': [hidden_layer_sizes],  # Capas ocultas pasadas como argumento
        'activation': ['relu'],  # Fijo
        'solver': ['adam'],  # Fijo
        'alpha': [0.001],  # Regularización L2 fija
        'learning_rate_init': [learning_rate_init],  # Tasa de aprendizaje pasada como argumento
        'max_iter': [300],  # Fijo
    }

    # Optimización con GridSearchCV
    grid_search_mlp = GridSearchCV(
        MLPRegressor(random_state=42),
        param_grid_mlp,
        cv=5,  # Validación cruzada con 5 folds
        scoring='neg_mean_squared_error',
        verbose=1,
        n_jobs=-1  # Paralelización
    )

    # Entrenar el modelo con los datos escalados
    grid_search_mlp.fit(X_train, y_train)

    # Mejor modelo encontrado
    best_mlp_model = grid_search_mlp.best_estimator_
    print(f"Mejores parámetros para hidden_layer_sizes={hidden_layer_sizes}, learning_rate_init={learning_rate_init}: {grid_search_mlp.best_params_}")

    # Validación cruzada en el conjunto de entrenamiento
    cv_scores_mse = cross_val_score(best_mlp_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_mean_mse = -cv_scores_mse.mean()  # MSE promedio (convertido a positivo)
    cv_std_mse = cv_scores_mse.std()  # Desviación estándar del MSE
    cv_rmse = np.sqrt(cv_mean_mse)  # RMSE promedio

    print("\nValidación cruzada para Redes Neuronales:")
    print(f"Error Cuadrático Medio (MSE) promedio en validación cruzada: {cv_mean_mse:.4f}")
    print(f"Raíz del Error Cuadrático Medio (RMSE) promedio en validación cruzada: {cv_rmse:.4f}")
    print(f"Desviación estándar del MSE en validación cruzada: {cv_std_mse:.4f}")

    # Generar predicciones para el conjunto de prueba
    y_pred = best_mlp_model.predict(X_test)

    # Guardar predicciones en archivo CSV dentro de la carpeta `predictions/RedesNeuronales`
    predictions_file = f"predictions/RedesNeuronales/predicciones_hidden_{hidden_layer_sizes}_lr_{learning_rate_init}.csv"
    pd.DataFrame({
        "Id": range(len(y_pred)),
        "Predicted_Rating": y_pred
    }).to_csv(predictions_file, index=False)
    print(f"Predicciones guardadas en '{predictions_file}'.")

    # Guardar métricas en archivo CSV dentro de `metrics/RedesNeuronales`
    metrics_file = f"metrics/RedesNeuronales/metrics_hidden_{hidden_layer_sizes}_lr_{learning_rate_init}.csv"
    metrics_table = pd.DataFrame({
        "Métrica": ["MSE promedio (CV)", "RMSE promedio (CV)", "Desviación estándar (MSE)"],
        "Valor": [cv_mean_mse, cv_rmse, cv_std_mse]
    })
    metrics_table.to_csv(metrics_file, index=False)
    print(f"Métricas guardadas en '{metrics_file}'.")

    return cv_mean_mse, metrics_file, best_mlp_model

# Proceso principal para iterar sobre los hiperparámetros
def main_nn(X_train, y_train, X_test):
    results = []
    hidden_layer_sizes_values = [(50,), (50, 50), (100, 50)]
    learning_rate_values = [0.001, 0.01]

    # Iterar sobre diferentes combinaciones de hidden_layer_sizes y learning_rate_init
    for hidden_layer_sizes in hidden_layer_sizes_values:
        for learning_rate_init in learning_rate_values:
            cv_mean_mse, metrics_file, model = train_and_evaluate_nn(
                X_train, y_train, X_test, hidden_layer_sizes, learning_rate_init
            )
            results.append((hidden_layer_sizes, learning_rate_init, cv_mean_mse, metrics_file, model))

    # Seleccionar el mejor modelo basado en MSE promedio
    best_model = min(results, key=lambda x: x[2])
    print("\nMejor modelo:")
    print(f"hidden_layer_sizes: {best_model[0]}, learning_rate_init: {best_model[1]}, MSE promedio: {best_model[2]:.4f}")

    # Guardar métricas del mejor modelo en `metrics/MejoresModelos/NN_mejor_modelo.csv`
    best_metrics_file = "metrics/MejoresModelos/NN_mejor_modelo.csv"
    pd.read_csv(best_model[3]).to_csv(best_metrics_file, index=False)
    print(f"Métricas del mejor modelo guardadas en '{best_metrics_file}'.")

    return best_model

# Escalar los datos (muy importante para redes neuronales)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Llamar al proceso principal
best_model = main_nn(X_train_scaled, y_train, X_test_scaled)

# Exportacion de predicciones